In [1]:
import numpy as np
import pandas as pd
from tkinter import *
from tkinter import colorchooser

In [2]:
class Utils():
    def __init__(self):
        None
    
    @staticmethod
    def makeDefaultWindow(title, geometry):
        window = Tk()
        window.geometry(geometry)
        window.title(title)
        window.resizable(width=False, height=False)
        return window
    
    @staticmethod
    def create_label(window, default_text, font_size, bg='default'):
        if bg == 'default':
            bg = window.cget('bg')

        label = Label(window, text=default_text, bg=bg)
        label.config(font=("Courier", font_size))
        return label

    @staticmethod
    def create_textbox(window, height, width, default_text, font_size):
        textbox = Text(window, height=height, width=width)
        textbox.insert(1.0, default_text)
        textbox.configure(font=("Courier", font_size))
        return textbox
    @staticmethod
    def get_textbox_to_num(textbox):
        txt = textbox.get('1.0', 'end').replace('\n', '')
        return int(txt)

    @staticmethod
    def get_textbox(textbox):
        txt = textbox.get('1.0', 'end').replace('\n', '')
        return txt

    @staticmethod
    def modify_textbox(textbox, txt):
        textbox.delete('1.0', 'end')
        textbox.insert(1.0, txt)

In [3]:
class GridModule():
    def __init__(self, window, parent, grid_x, grid_y, grid_width, grid_height, cnt_grid_x = 10, cnt_grid_y = 10, 
                 max_grid = 30, min_grid = 2):
        
        self.window = window
        self.parent = parent
        
        self.grid_x = grid_x
        self.grid_y = grid_y
        self.grid_width = grid_width
        self.grid_height = grid_height
        
        self.cnt_grid_x = cnt_grid_x
        self.cnt_grid_y = cnt_grid_y
        self.recWidth = self.grid_width / self.cnt_grid_x
        self.recHeight = self.grid_height / self.cnt_grid_y
        ㅅ
        self.max_grid = max_grid
        self.min_grid = min_grid        
        
        self.canvas = None
        self.make_grid()
        
    def make_grid(self):
        if self.canvas != None:
            self.canvas.destroy()
            
        self.canvas = Canvas(self.window, width=self.grid_width + 20, height = self.grid_height + 20)
        self.canvas.place(x=self.grid_x, y=self.grid_y)
        
        for i in range(self.cnt_grid_x):
            for j in range(self.cnt_grid_y):
                self.canvas.create_rectangle(i*self.recWidth+5, j*self.recHeight+5, (i+1)*self.recWidth+5,
                                                      (j+1)*self.recHeight+5, fill='white', tags='grid')
        
    def submit(self, furniture_objlist, grid_x_textbox, grid_y_textbox):
        for f in furniture_objlist:
            f.destroy()
            
        self.parent.furniture_objlist = []
            
        self.cnt_grid_x = Utils.get_textbox_to_num(grid_x_textbox)
        self.cnt_grid_y = Utils.get_textbox_to_num(grid_y_textbox)
        
        if self.cnt_grid_x > self.max_grid:
            self.cnt_grid_x = self.max_grid
            Utils.modify_textbox(grid_x_textbox, str(self.max_grid))
        elif self.cnt_grid_x < self.min_grid:
            self.cnt_grid_x = self.min_grid
            Utils.modify_textbox(grid_y_textbox, str(self.min_grid))
            
        if self.cnt_grid_y > self.max_grid:
            self.cnt_grid_y = self.max_grid
            Utils.modify_textbox(grid_y_textbox, str(self.max_grid))
        elif self.cnt_grid_y < self.min_grid:
            self.cnt_grid_y = self.min_grid
            Utils.modify_textbox(grid_y_textbox, str(self.min_grid))
            
        self.recWidth = self.grid_width / self.cnt_grid_x
        self.recHeight = self.grid_height / self.cnt_grid_y
        self.make_grid()
        
        self.parent.focus_object = None
        
    def get_rect_size(self):
        return (self.grid_x, self.grid_y, self.grid_width, self.grid_height, self.recWidth, self.recHeight)

In [4]:
class ListModule():
    def __init__(self, window, parent, list_x, list_y, list_width, list_height, bg='cyan'):
        self.furniture_list = pd.read_csv('color.csv')
        self.parent = parent
        self.canvas = Canvas(window, width=list_width, height=list_height, highlightthickness=0, bg=bg)
        self.canvas.place(x=list_x, y=list_y)
        
        self.frame = Frame(window)
        self.frame.grid(row=0, column=0, sticky='n')
        
        rows = 8
        columns = 2
        n=0
        for i in range(0, rows):
            for j in range(0, columns):
                self.make_button(canvas=self.canvas, n = n, row=i, column=j)
                n+=1
                
        self.frame.grid_columnconfigure(1, minsize=20)
    
    def make_button(self, canvas, n, row, column):
        text = self.furniture_list['Type'][n]
        bg = self.furniture_list['Color'][n]
        active = self.furniture_list['ActiveColor'][n]
        font_color = self.furniture_list['TextColor'][n]
        
        btn = Button(canvas, text=text, command=lambda:self.click(btn), width=15, height=4)
        btn.config(font=('Courier', 14), bg=bg, activebackground=active, fg=font_color, activeforeground=font_color)
        btn.grid(row=row, column=column, sticky='we')
    
    def click(self, button):
        focus = self.parent.focus_object
        focus_object = focus.get_object()
        
        text = button.cget('text')
        color = button.cget('bg')
        font_color = button.cget('fg')
        
        focus_object.config(bg=color)
        focus_object.itemconfig(focus_object.find_withtag(focus.text), text=text, fill=font_color)
        
        #text 크기도 오브젝트 크기에 맞춰서 변경가능하도록

In [5]:
class CreateModule():
    def __init__(self, window, parent, create_x, create_y, create_width, create_height, bg='cyan'):
        self.canvas = Canvas(window, width=create_width, height=create_height, bg=bg, highlightthickness=0)
        self.canvas.place(x=create_x, y=create_y)
        
        self.color_code = ((255,255,255), '#ffffff')
        self.make_create_layout(parent)
        
    def choose_color(self, color_rect):
        self.color_code = colorchooser.askcolor(title='Choose Color')
        #self.canvas.itemconfig(self.canvas.find_withtag(color_rect), fill=self.color_code[1])
        self.canvas.itemconfig(self.canvas.find_withtag(color_rect), fill='cyan')
    
    def make_create_layout(self, parent):
        label = Utils.create_label(window=self.canvas, default_text='Color', font_size=20)
        label.place(x=30, y=80)
        
        color_rect = self.canvas.create_rectangle(290, 80, 330, 115, fill='cyan')#self.color_code[1])
        color_button = Button(self.canvas, text='Select', height=1, width=13,
                             command = lambda:self.choose_color(color_rect))
        color_button.place(x=125, y=80)
        color_button.configure(font=("Courier", 14))
        
        label = Utils.create_label(window=self.canvas, default_text='Size', font_size=20)
        label.place(x=30, y=150)
        label = Utils.create_label(window=self.canvas, default_text='X', font_size=20)
        label.place(x=150, y=150)
        label = Utils.create_label(window=self.canvas, default_text='X', font_size=20)
        label.place(x=215, y=150)
        
        size_x_textbox = Utils.create_textbox(window=self.canvas, height=1, width=2, default_text='1', font_size=20)
        size_y_textbox = Utils.create_textbox(window=self.canvas, height=1, width=2, default_text='1', font_size=20)
        size_z_textbox = Utils.create_textbox(window=self.canvas, height=1, width=2, default_text='1', font_size=20)
        size_x_textbox.place(x=110, y=150)
        size_y_textbox.place(x=175, y=150)
        size_z_textbox.place(x=240, y=150)
        
        create_btn = Button(self.canvas, text='Create', width=19, height=1, bg='cyan',
                           command=lambda: parent.create_furniture(self.color_code, size_x_textbox, size_y_textbox, size_z_textbox))
        create_btn.configure(font=("Courier" ,20))
        create_btn.place(x=20, y=0)

In [6]:
class Furniture():
    def __init__(self, window, parent, bg, size, xlist, ylist, recWidth, recHeight, x, y, grid_pos):
        width = (size[0] * recWidth)-1
        height = (size[1] * recHeight)-1
        
        self.position = [x, y]
        self.parent = parent
        self.window = window
        self.recWidth = recWidth
        self.recHeight = recHeight
        self.obj = Canvas(window, bg=bg, width=width, height=height, highlightthickness=0)
        self.obj.place(x=x, y=y)
        self.obj.bind('<B1-Motion>', lambda event, recWidth=recWidth, recHeight=recHeight: 
                      self.move(event, recWidth, recHeight))
        self.obj.bind('<ButtonRelease-1>', lambda event, xlist=xlist, ylist=ylist, size_x=size[0], 
                      size_y=size[1], grid_pos=grid_pos: self.release(event, xlist, ylist, size[0], size[1], grid_pos))
        self.obj.bind('<Button-1>', self.focus_this)
        
        self.text_pos = [width/2, height/2]
        self.text = self.obj.create_text(self.text_pos[0], self.text_pos[1], fill='black', text='')
        
    def get_object(self):
        return self.obj
    
    def get_textposition(self):
        text_x = self.position[0] + self.recWidth/2
        text_y = self.position[1] + self.recHeight/2
        
        return text_x, text_y
    
    def focus_this(self, event):       
        mouse_x = self.window.winfo_pointerx() - self.window.winfo_rootx()
        mouse_y = self.window.winfo_pointery() - self.window.winfo_rooty()
        min_x = self.parent.grid_pos[0]
        min_y = self.parent.grid_pos[1]
        max_x = self.parent.grid_pos[2]
        max_y = self.parent.grid_pos[3]
        
        if (mouse_x <= max_x and mouse_x >= min_x) and (mouse_y <= max_y and mouse_y >= min_y):        
            for furniture in self.parent.furniture_objlist:
                furniture.config(bg='cyan')
            self.parent.focus_object = self
            self.obj.config(bg='lightblue')
        
    def move(self, event, recWidth, recHeight):        
        mouse_x = self.window.winfo_pointerx() - self.window.winfo_rootx()
        mouse_y = self.window.winfo_pointery() - self.window.winfo_rooty()
        event.widget.place(x=int(mouse_x-self.recWidth/2), y=int(mouse_y-self.recHeight/2))
        
    def release(self, event, xlist, ylist, size_x, size_y, grid_pos):
        mouse_x = self.window.winfo_pointerx() - self.window.winfo_rootx()
        mouse_y = self.window.winfo_pointery() - self.window.winfo_rooty()
        
        min_x = grid_pos[0]
        min_y = grid_pos[1]
        max_x = grid_pos[2]
        max_y = grid_pos[3]
        
        if mouse_x > max_x or mouse_y > max_y or mouse_x < min_x or mouse_y < min_y:
            event.widget.place(x=mouse_x-self.recWidth/2, y=mouse_y-self.recHeight/2)
            self.position = [mouse_x-self.recWidth/2, mouse_y-self.recHeight/2]
        
        else:
            place_x = xlist[0]
            for x in xlist:
                if mouse_x > x:
                    place_x = x
                    
            place_y = ylist[0]
            for y in ylist:
                if mouse_y > y:
                    place_y = y
             
            for x in reversed(xlist):
                if place_x + self.recWidth * size_x > max_x+5:
                    place_x = x
                    
            for y in reversed(ylist):
                if place_y + self.recHeight * size_y > max_y+5:
                    place_y = y

            event.widget.place(x=place_x+1, y=place_y+1)
            self.position = [place_x + 1, place_y + 1]

In [7]:
class Application():
    def __init__(self, title='Furniture Layout Batch Application', 
                 window_width=1500, window_height=1000, window_x=200, window_y=200):
        geometry = str(window_width) + 'x' + str(window_height) + '+' + str(window_x) + '+' + str(window_y)
        self.window = Utils.makeDefaultWindow(title=title, geometry=geometry)
        
        self.grid_module=  GridModule(self.window, parent = self, grid_x = 450, grid_y = 200, grid_width=600, grid_height=600)
        self.list_module = ListModule(self.window, parent = self, list_x = 50, list_y = 80, 
                              list_width=350, list_height=800, bg='cyan')
        self.create_module = CreateModule(self.window, parent=self, create_x = 1100, create_y = 80, 
                                          create_width=350, create_height=800, bg='cyan')
        
        self.furniture_objlist = []
        self.furniture_list = {}
        self.grids = []
        
        self.focus_object = None
        
        self.create_resize_grid_layout()
        
        #x1, y1, x2, y2
        self.grid_pos = [450, 200, 1050, 800]
        self.list_pos = [50, 80, 400, 880]
        self.create_pos = [1100, 80, 1450, 880]        
        
    def runApp(self):
        self.window.mainloop()
        
    def create_resize_grid_layout(self):
        label = Utils.create_label(window=self.window, default_text = 'RoomSize', font_size=20)
        label.place(x=680, y=50)
        
        label = Utils.create_label(window=self.window, default_text='X', font_size=20)
        label.place(x=730, y=100)
        
        grid_x_textbox = Utils.create_textbox(window=self.window, height=1, width=7, default_text='10', font_size=20)
        grid_x_textbox.place(x=610, y=100)
        
        grid_y_textbox = Utils.create_textbox(window=self.window, height=1, width=7, default_text='10', font_size=20)
        grid_y_textbox.place(x=755, y=100)
        
        submit_bt = Button(self.window, text='Submit', command = lambda: self.grid_module.submit(self.furniture_objlist,
                                                                                                grid_x_textbox,
                                                                                                grid_y_textbox))
        submit_bt.configure(font=("Courier", 14))
        submit_bt.place(x=900, y=98)
        
    def add_dict(self, color, size_list):
        key = 'furniture ' + str(len(self.furniture_list)+1)
        self.furniture_list[key] = {}
        
        size = str(size_list[0]) + 'x' + str(size_list[1]) + 'x' + str(size_list[2])
        self.furniture_list[key]['color'] = color
        self.furniture_list[key]['size'] = size
        
    def create_furniture(self, color_code, x_textbox, y_textbox, z_textbox):
        color = color_code[1]
        size_x = Utils.get_textbox_to_num(x_textbox)
        size_y = Utils.get_textbox_to_num(y_textbox)
        size_z = Utils.get_textbox_to_num(z_textbox)
        
        self.add_dict(color, size_list=(size_x, size_y, size_z))
        
        grid_x, grid_y, grid_width, grid_height, recWidth, recHeight = self.grid_module.get_rect_size()
        xlist = [x+grid_x+5 for x in range(0, grid_width, int(recWidth))]
        ylist = [y+grid_y+5 for y in range(0, grid_height, int(recHeight))]
            
        furniture = Furniture(window=self.window, parent = self, bg='cyan', size=(size_x, size_y), xlist=xlist, ylist=ylist,
                             recWidth=recWidth, recHeight=recHeight, x=grid_x-recWidth, y=grid_y-recHeight, grid_pos=self.grid_pos)
        self.furniture_objlist.append(furniture.get_object())
        self.focus_object = furniture

In [8]:
if __name__ == '__main__':
    app = Application(title='Furniture Batch Application', window_width=1500, window_height=1000, window_x = 200, window_y = 200)
    app.runApp()

In [10]:
#Focus Object -> Button -> 해당 걸로 선택됨
#색감 조정

#furniture_objlist에서 Furniture.Canvas 대신 Furniture로 바꾸고
#Active Color랑 Color 그 Furniture에 저장할 수 있도록 바꿔주기
#가능하면 색변경대신 Border 칼라 바꾸게

SyntaxError: invalid syntax (Temp/ipykernel_4460/3180503799.py, line 1)

### 원 그리기

In [212]:
'''import tkinter
import random


c = tkinter.Canvas(width = 400, height = 300)
c.pack()

class create_circles():
    def __init__(self, event):
        self.r = 50
        self.x, self.y = event.x, event.y
        self.color = '#{:06x}'.format(random.randrange(256 ** 3))
        self.circ = c.create_oval(self.x - self.r, self.y - self.r, self.x + self.r, self.y + self.r, fill=self.color)
        self.shrink()

    def shrink(self):
        if self.r > 0:
            self.r -= 1
            c.coords(self.circ, self.x-self.r, self.y-self.r, self.x+self.r, self.y+self.r)
            c.after(100, self.shrink)

c.bind('<Button-1>', create_circles)
tkinter.mainloop()
'''

In [226]:
'''
def circles(e):
    print("On")

c = tkinter.Canvas(width=400, height=300)
c.pack()
hblue = "#%02x%02x%02x" % (0, 0, 255)
r=50
x,y = 100, 100
circle = c.create_oval(x-r, y-r, x+r, y+r, fill='white')
c.tag_bind(circle, '<Button-1>', circles)

tkinter.mainloop()
'''

On
On
On
On
On
On
